In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint
from inpainting.custom_layers import Reshape
from inpainting.losses import r2_masked_batch_loss, r2_total_batch_loss, nll_masked_batch_loss, _nll_masked_batch_loss
from inpainting.inpainters.digits import DigitsLinearInpainter
from pathlib import Path

Dupa dupa dupa

In [ ]:
from inpainting.datasets.digits import train_val_datasets
from inpainting.visualizations.digits import digit_with_mask as vis_digit_mask
from inpainting.training import train_inpainter
from inpainting.utils import classifier_experiment, inpainted
from inpainting import losses2 as l2

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = "white"

In [ ]:
# !ps aux | grep mprzewie

In [ ]:
# !echo $CUDA_VISIBLE_DEVICES
# !nvidia-smi

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

In [ ]:
experiment_path = Path("../results/digits/more_visualizations")
experiment_path.mkdir(exist_ok=True)

In [ ]:
ds_train, ds_val = train_val_datasets()

fig, axes = plt.subplots(10, 10, figsize=(15, 15))
for i in range(100):
    (x,j), y = ds_train[i]
    ax = axes[i // 10, i%10]
    ax.set_title(f"{y}")
    vis_digit_mask(x, j,ax)
train_fig = plt.gcf()
train_fig.savefig(experiment_path / "train.png")
plt.show()

In [ ]:
classifier = MLPClassifier((100, 200, 10,), learning_rate_init=4e-3, max_iter=1000).fit(ds_train.X.reshape(-1, 64), ds_train.y)

In [ ]:
batch_size=8
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_val = DataLoader(ds_val, batch_size, shuffle=True)

In [ ]:
m_std = lambda x, j, p, m, a, d: m.std(dim=0).mean()

In [ ]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

inpainter = DigitsLinearInpainter(bias=True, n_hidden_layers=3, n_mixes=1)
opt = optim.Adam(inpainter.parameters(), lr=4e-3, weight_decay=0.)
n_epochs = 3
history = train_inpainter(
    inpainter, 
    dl_train, 
    dl_val, 
    opt, 
#     loss_fn=l2.nll_buffered(9),
    loss_fn=l2.nll_buffered,
#     loss_fn=l2.nll_zero,
    n_epochs=n_epochs,
    losses_to_log=dict(
        nll_zero=l2.nll_zero,
        nll_buffered=l2.nll_buffered
        
    ),
    device=device
)

In [ ]:
[h["losses"] for h in history]

In [ ]:
# [
#     (h["losses"]["objective"]["train"] - h["losses"]["nll_ung"]["train"], h["losses"]["objective"]["train"] - h["losses"]["nll_ancient"]["train"])

#     for h in history
# ]

In [ ]:
with (experiment_path / "inpainter.schema").open("w") as f:
    print(inpainter, file=f)

# torch.save(inpainter.state_dict, experiment_path / "inpainter.state")

In [ ]:
history[-1]["losses"]["objective"]

In [ ]:
history_tmp = history
history = history

In [ ]:
for loss_name in set(history[0]["losses"].keys()):
    for fold in ["train", "val"]:
        
        plt.plot(
            list(range(len(history))),
            [h["losses"][loss_name][fold] for h in history],
            label=fold
        )
    plt.title(loss_name)
    plt.legend()
    fig = plt.gcf()
    fig.savefig(experiment_path / f"history.{loss_name}.png")
    plt.show()

In [ ]:
import inpainting.visualizations.samples as vis

In [ ]:
x = list(zip(*history[0]["sample_results"]["train"]))[0]


In [ ]:
skip = 10

row_length = vis.row_length(*list(zip(*history[0]["sample_results"]["train"]))[0])
fig, axes = plt.subplots(
    int(np.ceil(len(history) / skip)* 2), 
    row_length,
    figsize=(20, 30)
)


for e, h in enumerate(history):
    if e % skip !=0 and e != (len(history) -1):
        continue
    
    for ax_no, fold in [(0,"train"), (1,"val")]:
        x, j, p, m, a, d, y = [t[0] for t in  h["sample_results"][fold]]
        row_no = (e // skip)*2 + ax_no

        vis.visualize_sample(
            x, j, p, m, a, d, y, 
            ax_row=axes[row_no], 
            title_prefixes={
                0: f"{e} {fold} ",
#                 1: f"y_m = {y_masked_pred}"
            }
        )

epochs_fig = plt.gcf()
epochs_fig.savefig(experiment_path / "epochs_renders.png")

In [ ]:
epochs_path = experiment_path / "epochs"
epochs_path.mkdir(exist_ok=True)
skip = 20
n_rows = 16

for e, h in enumerate(history):
    if e % skip !=0 and e != (len(history) -1):
        continue
    
    for ax_no, fold in [(0,"train"), (1,"val")]:
        
        
        row_length = vis.row_length(*list(zip(*h["sample_results"][fold]))[0])

        fig, axes = plt.subplots(
            n_rows, 
            row_length,
            figsize=(20, 30)
        )

        for row_no, (x, j, p, m ,a, d, y) in enumerate(list(zip(*h["sample_results"][fold]))[:n_rows]):
            vis.visualize_sample(
                x, j, p, m, a, d, y, 
                ax_row=axes[row_no], 
                title_prefixes={
                    0: f"{e} {fold} ",
#                     1: f"y_m = {y_masked_pred}"
                }
            )
        
        title = f"{e}_{fold}"
        plt.suptitle(title)
        plt.savefig(epochs_path / f"{title}.png")
#         plt.show()
            

# epochs_fig = plt.gcf()
# epochs_fig.savefig(experiment_path / "epochs_renders.png")

# Sampling from the inpainter

In [ ]:
samples_path = experiment_path / "samples"
samples_path.mkdir(exist_ok=True)


samples = history[-1]["sample_results"]


n_rows = 16

for ax_no, fold in [(0,"train"), (1,"val")]:

    fig, axes = plt.subplots(
        n_rows, 
        3 + 3 * inpainter.n_mixes,
        figsize=(20, 30)
    )
    X, J, P, M, A, D, Y = samples[fold]
    P_, M_, A_, D_ = inpainter(
        torch.tensor(X).to(device),
        torch.tensor(J).to(device)
    )
    P_n, M_n, A_n, D_n = [
        t.detach().cpu().numpy()
        for t in 
        [P_, M_, A_, D_]
    ]
    
    for row_no, (x, j, p, m ,a, d, y) in enumerate(list(zip(X, J, P_n, M_n, A_n, D_n, Y))[:n_rows]):
        vis.visualize_distribution_samples(
            x, j, p, m, a, d, y, 
            axes[row_no]
        )
        